In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow import keras

In [2]:
df=pd.read_csv(r"C:\Users\soura\Desktop\csv files\insurance_data_codebasics.csv")

In [3]:
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
x_train,x_test,y_train,y_test=train_test_split(df[["age","affordibility"]],df["bought_insurance"],test_size=0.2, random_state=25)

In [6]:
len(x_train)

22

In [7]:
x_train_scaled=x_train.copy()
x_train_scaled['age'] = x_train_scaled['age']/100

x_test_scaled = x_test.copy()
x_test_scaled['age']=x_test_scaled['age']/100

In [8]:
x_train_scaled

,age,affordibility
0,0.22,1
13,0.29,0
6,0.55,0
17,0.58,1
24,0.50,1
19,0.18,1
25,0.54,1
16,0.25,0
20,0.21,1
3,0.52,0


In [10]:
model=keras.Sequential([
    keras.layers.Dense(1,input_shape=(2,), activation='sigmoid' , kernel_initializer= 'ones' , bias_initializer= 'zeros')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

model.fit(x_train_scaled,y_train, epochs=5000)

Epoch 1/5000
1/1 [==============================] - 11s 11s/step - loss: 0.7113 - accuracy: 0.5000
Epoch 2/5000
1/1 [==============================] - 0s 45ms/step - loss: 0.7110 - accuracy: 0.5000
Epoch 3/5000
1/1 [==============================] - 0s 49ms/step - loss: 0.7106 - accuracy: 0.5000
Epoch 4/5000
1/1 [==============================] - 0s 50ms/step - loss: 0.7102 - accuracy: 0.5000
Epoch 5/5000
1/1 [==============================] - 0s 45ms/step - loss: 0.7098 - accuracy: 0.5000
Epoch 6/5000
1/1 [==============================] - 0s 44ms/step - loss: 0.7094 - accuracy: 0.5000
Epoch 7/5000
1/1 [==============================] - 0s 46ms/step - loss: 0.7091 - accuracy: 0.5000
Epoch 8/5000
1/1 [==============================] - 0s 45ms/step - loss: 0.7087 - accuracy: 0.5000
Epoch 9/5000
1/1 [==============================] - 0s 49ms/step - loss: 0.7083 - accuracy: 0.5000
Epoch 10/5000
1/1 [==============================] - 0s 42ms/step - loss: 0.7079 - accuracy: 0.5000
Epoch 11/

In [11]:
model.evaluate(x_test_scaled,y_test)

1/1 [==============================] - 3s 3s/step - loss: 0.3550 - accuracy: 1.0000


[0.3549775183200836, 1.0]

In [12]:
x_test_scaled

,age,affordibility
2,0.47,1
10,0.18,1
21,0.26,0
11,0.28,1
14,0.49,1
9,0.61,1


In [13]:
model.predict(x_test_scaled)

1/1 [==============================] - 2s 2s/step


array([[0.70548487],
       [0.35569552],
       [0.16827849],
       [0.47801173],
       [0.7260697 ],
       [0.82949835]], dtype=float32)

In [14]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

In [15]:
coef,intercept = model.get_weights()
coef,intercept

(array([[5.060867 ],
        [1.4086502]], dtype=float32),
 array([-2.9137027], dtype=float32))

In [18]:
# now crosscheck of predicted model value by raw method

def sigmoid(x):
    import math
    return 1 / (1 + math.exp(-x))

In [19]:
def prediction_function(age, affordibility):
    weighted_sum = coef[0]*age + coef[1]*affordibility + intercept
    return sigmoid(weighted_sum)

In [20]:
def log_loss(y_true,y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,(1-epsilon)) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new) + (1-y_true)*np.log(1-y_predicted_new))

In [21]:
def sigmoid_numpy(x):
    return 1/(1+np.exp(-x))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [22]:
# prediction model works properly

prediction_function(.47, 1)

C:\Users\soura\AppData\Local\Temp\ipykernel_4228\3236369613.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1 / (1 + math.exp(-x))


0.7054848693136117

In [37]:
def gradient_descent(age, affordibility, y_true, epochs, loss_thresold):
    # w1, w2, bias
    w1=w2=1
    bias=0
    learning_rate=0.5
    n=len(age)

    for i in range(epochs):
        weighted_sum = w1*age + w2*affordibility + bias
        y_predicted = sigmoid_numpy(weighted_sum)

        loss = log_loss(y_true, y_predicted)
    
        w1d = (1/n)*np.dot(np.transpose(age), (y_predicted-y_true))
        w2d = (1/n)*np.dot(np.transpose(affordibility),(y_predicted-y_true))
    
        bias_d = np.mean(y_predicted-y_true)
    
        w1= w1- learning_rate*w1d
        w2 = w2 - learning_rate*w2d
        bias = bias - learning_rate + bias_d
    
        print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

        if loss<=loss_thresold:
            break
    return w1, w2, bias

In [38]:
gradient_descent(x_train_scaled['age'], x_train_scaled['affordibility'],y_train,1000,0.5321)

Epoch:0, w1:0.974907633470177, w2:0.948348125394529, bias:-0.27316264527262835, loss:0.7113403233723417
Epoch:1, w1:0.9620300855866085, w2:0.915063748814626, bias:-0.6082254440416607, loss:0.6522828988560008
Epoch:2, w1:0.9649149991849144, w2:0.9060870343188192, bias:-1.0224949874944445, loss:0.6085289626774781
Epoch:3, w1:0.9874959162061404, w2:0.9283988150631846, bias:-1.5346870619948463, loss:0.5932298838909815
Epoch:4, w1:1.0330500528533009, w2:0.9885107079147643, bias:-2.159581941311168, loss:0.6252554540616029
Epoch:5, w1:1.1024200304274125, w2:1.0893858295415275, bias:-2.899483203368081, loss:0.7254084719207098
Epoch:6, w1:1.192718331197512, w2:1.2275431325233526, bias:-3.7387376297900117, loss:0.9063731959059237
Epoch:7, w1:1.2982774978565368, w2:1.3938881455911416, bias:-4.64924869924984, loss:1.1633292373978399
Epoch:8, w1:1.4132517343338946, w2:1.5781531493330572, bias:-5.603077554407714, loss:1.4768477182282593
Epoch:9, w1:1.5333776226074383, w2:1.7724591929689812, bias:-6.

C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core

Epoch:733, w1:92.19316606642624, w2:149.85364955780955, bias:-730.5593913927497, loss:17.26938819745534
Epoch:734, w1:92.31839333915352, w2:150.05819501235501, bias:-731.5593913927497, loss:17.26938819745534
Epoch:735, w1:92.44362061188079, w2:150.26274046690048, bias:-732.5593913927497, loss:17.26938819745534
Epoch:736, w1:92.56884788460806, w2:150.46728592144595, bias:-733.5593913927497, loss:17.26938819745534
Epoch:737, w1:92.69407515733533, w2:150.67183137599142, bias:-734.5593913927497, loss:17.26938819745534
Epoch:738, w1:92.8193024300626, w2:150.87637683053688, bias:-735.5593913927497, loss:17.26938819745534
Epoch:739, w1:92.94452970278988, w2:151.08092228508235, bias:-736.5593913927497, loss:17.26938819745534
Epoch:740, w1:93.06975697551715, w2:151.28546773962782, bias:-737.5593913927497, loss:17.26938819745534
Epoch:741, w1:93.19498424824442, w2:151.4900131941733, bias:-738.5593913927497, loss:17.26938819745534
Epoch:742, w1:93.3202115209717, w2:151.69455864871875, bias:-739.5

C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core

Epoch:768, w1:96.57612061188078, w2:157.0127404669009, bias:-765.5593913927497, loss:17.26938819745534
Epoch:769, w1:96.70134788460805, w2:157.21728592144638, bias:-766.5593913927497, loss:17.26938819745534
Epoch:770, w1:96.82657515733533, w2:157.42183137599184, bias:-767.5593913927497, loss:17.26938819745534
Epoch:771, w1:96.9518024300626, w2:157.6263768305373, bias:-768.5593913927497, loss:17.26938819745534
Epoch:772, w1:97.07702970278987, w2:157.83092228508278, bias:-769.5593913927497, loss:17.26938819745534
Epoch:773, w1:97.20225697551714, w2:158.03546773962825, bias:-770.5593913927497, loss:17.26938819745534
Epoch:774, w1:97.32748424824442, w2:158.2400131941737, bias:-771.5593913927497, loss:17.26938819745534
Epoch:775, w1:97.45271152097169, w2:158.44455864871918, bias:-772.5593913927497, loss:17.26938819745534
Epoch:776, w1:97.57793879369896, w2:158.64910410326465, bias:-773.5593913927497, loss:17.26938819745534
Epoch:777, w1:97.70316606642623, w2:158.85364955781012, bias:-774.55

...

Epoch:966, w1:121.37112061188074, w2:197.51274046690347, bias:-963.5593913927497, loss:17.26938819745534
Epoch:967, w1:121.49634788460801, w2:197.71728592144893, bias:-964.5593913927497, loss:17.26938819745534
Epoch:968, w1:121.62157515733529, w2:197.9218313759944, bias:-965.5593913927497, loss:17.26938819745534
Epoch:969, w1:121.74680243006256, w2:198.12637683053987, bias:-966.5593913927497, loss:17.26938819745534
Epoch:970, w1:121.87202970278983, w2:198.33092228508534, bias:-967.5593913927497, loss:17.26938819745534
Epoch:971, w1:121.9972569755171, w2:198.5354677396308, bias:-968.5593913927497, loss:17.26938819745534
Epoch:972, w1:122.12248424824438, w2:198.74001319417627, bias:-969.5593913927497, loss:17.26938819745534
Epoch:973, w1:122.24771152097165, w2:198.94455864872174, bias:-970.5593913927497, loss:17.26938819745534
Epoch:974, w1:122.37293879369892, w2:199.1491041032672, bias:-971.5593913927497, loss:17.26938819745534


C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core

Epoch:975, w1:122.4981660664262, w2:199.35364955781267, bias:-972.5593913927497, loss:17.26938819745534
Epoch:976, w1:122.62339333915347, w2:199.55819501235814, bias:-973.5593913927497, loss:17.26938819745534
Epoch:977, w1:122.74862061188074, w2:199.7627404669036, bias:-974.5593913927497, loss:17.26938819745534
Epoch:978, w1:122.87384788460801, w2:199.96728592144908, bias:-975.5593913927497, loss:17.26938819745534
Epoch:979, w1:122.99907515733528, w2:200.17183137599454, bias:-976.5593913927497, loss:17.26938819745534
Epoch:980, w1:123.12430243006256, w2:200.37637683054, bias:-977.5593913927497, loss:17.26938819745534
Epoch:981, w1:123.24952970278983, w2:200.58092228508548, bias:-978.5593913927497, loss:17.26938819745534
Epoch:982, w1:123.3747569755171, w2:200.78546773963095, bias:-979.5593913927497, loss:17.26938819745534
Epoch:983, w1:123.49998424824437, w2:200.9900131941764, bias:-980.5593913927497, loss:17.26938819745534
Epoch:984, w1:123.62521152097165, w2:201.19455864872188, bias:

C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\soura\anaconda3\envs\tensorflow\lib\site-packages\pandas\core

(125.50362061188073, 204.2627404669039, -996.5593913927497)